<a href="https://colab.research.google.com/github/SABDULLAH42/AIMK/blob/main/IP_Address.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [689 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 924 kB in 2s (610 kB/s)
Reading package lists... Done
sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2

In [2]:
df = spark.read.csv("sample_data/challenge.csv", header=True)
df.show()


+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [3]:
## Add a column to say yes or No to whether the Country is Mexico

import pyspark.sql.functions as sqlfunc
from pyspark.sql.functions import *
df = df.withColumn('home', when(df.Country =='Mexico', 'YES').otherwise('NO'))
df.show()

+---------------+--------------+-----------------+----------+----+
|     ip_address|       Country|      Domain Name|Bytes_used|home|
+---------------+--------------+-----------------+----------+----+
|  52.81.192.172|         China| odnoklassniki.ru|       463|  NO|
| 119.239.207.13|         China|         youtu.be|        51|  NO|
|  68.69.217.210|         China|        adobe.com|        10|  NO|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|  NO|
|   211.13.10.68|     Indonesia|          hud.gov|        29|  NO|
|   239.80.21.97|      Suriname|       smh.com.au|       218|  NO|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|  NO|
| 127.242.24.138|         China| surveymonkey.com|       123|  NO|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|  NO|
|   237.54.11.63|         China|       amazon.com|        83|  NO|
| 252.141.157.25|         Japan|      cornell.edu|       374|  NO|
|185.220.128.248|       Belgium|       weebly.com|       389| 

In [4]:
##Group by your new column and sum bytes_used

df2 = df.groupBy('home').agg(sqlfunc.sum(df.Bytes_used).alias('total_used'))
df2.show()

+----+----------+
|home|total_used|
+----+----------+
| YES|    6293.0|
|  NO|  508076.0|
+----+----------+



In [5]:
##Group by Country & use the sqlfunc.countDistinct function to calculate the number of IP address seen in EACH Country

df3 = df.groupBy('Country').agg(sqlfunc.countDistinct(df.ip_address).alias('number_of_ips'))
df3.sort(col("number_of_ips").desc()).show()

+--------------+-------------+
|       Country|number_of_ips|
+--------------+-------------+
|         China|          172|
|     Indonesia|          114|
|   Philippines|           65|
|        Russia|           56|
|        Brazil|           35|
|        Poland|           31|
|        Sweden|           28|
|         Japan|           25|
|Czech Republic|           23|
|      Portugal|           23|
|        France|           21|
|          Peru|           19|
|      Colombia|           17|
| United States|           15|
|     Argentina|           14|
|       Ukraine|           14|
|        Mexico|           13|
|      Thailand|           12|
|        Canada|           11|
|       Nigeria|           11|
+--------------+-------------+
only showing top 20 rows

